In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as rd

cols = ['Underlying']
strikeRange = [455,456,457,458,459,460,461]

for i in range(len(strikeRange)):
    cols += [f'Strike: ${strikeRange[i]}', f'call_openint {strikeRange[i]}', f'call_volume {strikeRange[i]}', f'call_bid {strikeRange[i]}',f'call_ask {strikeRange[i]}',f'put_bid {strikeRange[i]}',f'put_ask {strikeRange[i]}',f'put_volume {strikeRange[i]}',f'put_openint {strikeRange[i]}']

df = pd.DataFrame(columns=cols)


num_trades = 1200
for t in range(num_trades):
    if t > 0:
        df.loc[t,'Underlying'] = df.loc[t-1,'Underlying'] + np.random.normal(0,5)
    else:
        df.loc[t,'Underlying'] = 458
        for strike in range(len(strikeRange)):
            df.loc[t,f'call_openint {strikeRange[strike]}'] = round(np.random.normal(100,30))
            df.loc[t,f'put_openint {strikeRange[strike]}'] = round(np.random.normal(100,30))
            df.loc[t,f'call_volume {strikeRange[strike]}'] = round(np.random.normal(70,30))
            df.loc[t,f'put_volume {strikeRange[strike]}'] = round(np.random.normal(70,30))
            
        
    for strike in range(len(strikeRange)):
        df.loc[t,f'Strike: ${strikeRange[strike]}'] = f'Strike: ${strikeRange[strike]}'
        
        #get call data
        call_bid = np.random.normal(2,1)
        call_ask = call_bid-np.random.normal(0.2,0.1)
        df.loc[t,f'call_bid {strikeRange[strike]}'] = call_bid
        df.loc[t,f'call_ask {strikeRange[strike]}'] = call_ask
        
        #calculate put call parity to back out put bid/ask
        mean_call = (call_bid+call_ask)/2
        mean_put = mean_call-df.loc[t,'Underlying']+strikeRange[strike]

        #get put data
        put_bid = mean_put+np.random.normal(0.2,0.1)
        put_ask = mean_put-np.random.normal(0.2,0.1)
        df.loc[t,f'put_bid {strikeRange[strike]}'] = put_bid
        df.loc[t,f'put_ask {strikeRange[strike]}'] = put_ask

        #set volume and open interest
        if t>0:
            df.loc[t,f'call_openint {strikeRange[strike]}'] = max(df.loc[t-1,f'call_openint {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)
            df.loc[t,f'put_openint {strikeRange[strike]}'] = max(df.loc[t-1,f'put_openint {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)
            df.loc[t,f'call_volume {strikeRange[strike]}'] = max(df.loc[t-1,f'call_volume {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)
            df.loc[t,f'put_volume {strikeRange[strike]}'] = max(df.loc[t-1,f'put_volume {strikeRange[strike]}'] + round(np.random.normal(0,10)),0)


display(df)
df.to_csv('OptionsData1.csv')

,Underlying,Strike: $455,call_openint 455,call_volume 455,call_bid 455,call_ask 455,put_bid 455,put_ask 455,put_volume 455,put_openint 455,...,put_openint 460,Strike: $461,call_openint 461,call_volume 461,call_bid 461,call_ask 461,put_bid 461,put_ask 461,put_volume 461,put_openint 461
0,458,Strike: $455,129,20,1.808698,1.671215,-1.035993,-1.607006,67,28,...,129,Strike: $461,133,37,2.204422,1.958553,5.298257,4.857747,43,85
1,464.637916,Strike: $455,148,2,1.44953,1.074409,-8.229817,-8.5967,66,48,...,119,Strike: $461,144,28,0.771085,0.550017,-2.768828,-3.114248,43,86
2,468.062869,Strike: $455,138,14,1.828865,1.66756,-11.178917,-11.506307,64,49,...,126,Strike: $461,148,21,2.463426,2.267252,-4.50429,-4.955025,66,85
3,469.173757,Strike: $455,128,4,0.951704,0.793178,-13.269387,-13.378505,69,54,...,111,Strike: $461,168,16,2.424754,2.385012,-5.561128,-5.926589,71,93
4,459.926049,Strike: $455,143,0,2.683601,2.315566,-2.505555,-2.631636,58,65,...,116,Strike: $461,159,19,2.833359,2.709124,3.992753,3.651396,91,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,64.481455,Strike: $455,585,930,1.966971,1.7707,392.601924,392.149409,384,346,...,527,Strike: $461,187,827,3.211746,3.046393,400.001687,399.358872,409,177
1196,65.552616,Strike: $455,586,942,2.040649,1.766715,391.39722,391.130313,368,356,...,521,Strike: $461,186,836,1.364134,1.230814,397.104695,396.550511,411,174
1197,71.141383,Strike: $455,607,935,2.550117,2.467438,386.847347,386.177846,349,358,...,524,Strike: $461,192,831,1.765578,1.482024,391.657299,391.168476,413,168
1198,75.251971,Strike: $455,620,935,0.282255,0.054441,380.251263,379.772061,343,362,...,520,Strike: $461,205,836,2.158297,2.0294,387.961678,387.564836,409,160
